In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os

# Path to your data directory
data_dir = '/Users/advait/Desktop/Knee_Arthiritis_AI/data'
class_names = ["0Normal", "1Doubtful", "2Mild", "3Moderate", "4Severe"]

# Initialize lists to hold file paths and labels
file_paths = []
labels = []

# Load file paths and labels
for idx, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    for file_name in os.listdir(class_dir):
        if file_name.endswith(".png"):
            file_paths.append(os.path.join(class_dir, file_name))
            labels.append(idx)

# Convert to numpy arrays
file_paths = np.array(file_paths)
labels = np.array(labels)

# Perform stratified split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Convert numeric labels to string class names
train_labels = [class_names[label] for label in train_labels]
val_labels = [class_names[label] for label in val_labels]

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1.0 / 255.0
)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Create DataFrames for train and validation data
train_df = pd.DataFrame({'filename': train_paths, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_paths, 'class': val_labels})

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical',
    shuffle=True
)
validation_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)

# Load ResNet50 with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Build the top model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6, verbose=1)
model_checkpoint = ModelCheckpoint('best_resnet_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Calculate steps per epoch based on generator batch size
steps_per_epoch = len(train_paths) // train_generator.batch_size
validation_steps = len(val_paths) // validation_generator.batch_size

# Train model without EarlyStopping
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[reduce_lr, model_checkpoint],
    verbose=1
)


Found 1320 validated image filenames belonging to 5 classes.
Found 330 validated image filenames belonging to 5 classes.
Epoch 1/50


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-12-12 21:31:42.748126: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 3 of 8
2024-12-12 21:31:53.218547: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 6 of 8
2024-12-12 21:32:00.696782: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


165/165 ━━━━━━━━━━━━━━━━━━━━ 712s 4s/step - accuracy: 0.2787 - loss: 1.7780 - val_accuracy: 0.3140 - val_loss: 1.5469 - learning_rate: 1.0000e-04
Epoch 2/50


/opt/anaconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 1.5207 - learning_rate: 1.0000e-04
Epoch 3/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 111s 672ms/step - accuracy: 0.3139 - loss: 1.6038 - val_accuracy: 0.2927 - val_loss: 1.5368 - learning_rate: 1.0000e-04
Epoch 4/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 1.5789 - learning_rate: 1.0000e-04
Epoch 5/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 115s 697ms/step - accuracy: 0.2911 - loss: 1.6109 - val_accuracy: 0.2866 - val_loss: 1.5452 - learning_rate: 1.0000e-04
Epoch 6/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.9773 - learning_rate: 1.0000e-04
Epoch 7/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 114s 689ms/step - accuracy: 0.3253 - loss: 1.5496 - val_accuracy: 0.3140 - val_loss: 1.5286 - learning_rate: 1.0000e-04
Epoch 8/50
165/165 ━━━━━━━━━━━━

2024-12-12 21:56:40.276020: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


165/165 ━━━━━━━━━━━━━━━━━━━━ 1910s 12s/step - accuracy: 0.2758 - loss: 1.5430 - val_accuracy: 0.3140 - val_loss: 1.5204 - learning_rate: 9.0000e-06
Epoch 18/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 1.6542 - learning_rate: 9.0000e-06
Epoch 19/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 114s 691ms/step - accuracy: 0.2790 - loss: 1.5531 - val_accuracy: 0.3140 - val_loss: 1.5186 - learning_rate: 9.0000e-06
Epoch 20/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 1.9603 - learning_rate: 9.0000e-06
Epoch 21/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 560ms/step - accuracy: 0.3252 - loss: 1.5319
Epoch 21: ReduceLROnPlateau reducing learning rate to 2.6999998226528985e-06.
165/165 ━━━━━━━━━━━━━━━━━━━━ 116s 699ms/step - accuracy: 0.3252 - loss: 1.5319 - val_accuracy: 0.3140 - val_loss: 1.5203 - learning_rate: 9.0000e-06
Epoch 22/50
165/165 ━━━━━━━━━━━━━━━━━

2024-12-12 23:11:25.732124: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


165/165 ━━━━━━━━━━━━━━━━━━━━ 36567s 223s/step - accuracy: 0.3073 - loss: 1.5368 - val_accuracy: 0.3140 - val_loss: 1.5182 - learning_rate: 1.0000e-06
Epoch 50/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 1.9501 - learning_rate: 1.0000e-06


In [ ]:

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [20]:
# Get true labels from the validation generator
y_true_test = []
y_pred_test = []

batch_size = 8
# Make predictions and collect labels
for i in range(len(val_paths) // batch_size + 1):  # Process in batches
    x_batch, y_batch = next(validation_generator)
    y_true_test.extend(np.argmax(y_batch, axis=1))
    y_pred_batch = model.predict_on_batch(x_batch)
    y_pred_test.extend(np.argmax(y_pred_batch, axis=1))

y_true_test = np.array(y_true_test)
y_pred_test = np.array(y_pred_test)

cm_test = confusion_matrix(y_true_test, y_pred_test)
# Get true labels from the validation generator
y_true_train = []
y_pred_train = []

# Make predictions and collect labels
for i in range(len(train_paths) // batch_size + 1):  # Process in batches
    x_batch, y_batch = next(train_generator)
    y_true_train.extend(np.argmax(y_batch, axis=1))
    y_pred_batch = model.predict_on_batch(x_batch)
    y_pred_train.extend(np.argmax(y_pred_batch, axis=1))

y_true_train = np.array(y_true_train)
y_pred_train = np.array(y_pred_train)

cm_train = confusion_matrix(y_true_train, y_pred_train)

plt.figure(figsize=(6, 4))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Train Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

plt.figure(figsize=(6, 4))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Test Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


NameError: name 'confusion_matrix' is not defined

In [ ]:
classification_report_train = classification_report(y_true_train, y_pred_train, target_names= class_names)
print(classification_report_train)

classification_report_test = classification_report(y_true_test, y_pred_test, target_names= class_names)
print(classification_report_test)
